In [28]:
import os
# https://docs.python.org/3/library/urllib.request.html#urllib.request.urlopen
import urllib.request

import numpy as np
import tensorflow as tf

In [29]:
# Data sets
IRIS_TRAINING="./IRIS_data/iris_training.csv"
IRIS_TRAINING_URL="http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST="./IRIS_data/iris_test.csv"
IRIS_TEST_URL="http://download.tensorflow.org/data/iris_test.csv"

In [30]:
def main():
    # if the training and test sets aren't stored locally, download them.
    if not os.path.exists(IRIS_TRAINING):
        raw = urllib.request.urlopen(IRIS_TRAINING_URL).read()
        with open(IRIS_TRAINING, "wb") as f:
            f.write(raw)

    if not os.path.exists(IRIS_TEST):
        raw = urllib.request.urlopen(IRIS_TEST_URL).read()
        with open(IRIS_TEST, "wb") as f:
            f.write(raw)

    # Load datasets
    training_sets = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TRAINING,
        target_dtype=np.int,
        features_dtype=np.float32
    )
    test_sets = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TEST,
        target_dtype=np.int,
        features_dtype=np.float32
    )

    # Specify that all features have real-value data
    feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

    # Build 3 layer DNN with 10,20,10 units respectively
    classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="./models/iris_model")
    # Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": np.array(training_sets.data)},
        y=np.array(training_sets.target),
        num_epochs=None,
        shuffle=True
    )

    # Train model.
    classifier.train(input_fn=train_input_fn, steps=2000)

    # Define the test inputs
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": np.array(test_sets.data)},
        y=np.array(test_sets.target),
        num_epochs=1,
        shuffle=False
    )

    # Evaluate accuracy.
    accuracy=classifier.evaluate(input_fn=test_input_fn)
    print("accuracy: ",accuracy)
    
    accuracy_score = accuracy["accuracy"]

    print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

    # Classify two new flower samples.
    new_samples = np.array(
        [[6.4, 3.2, 4.5, 1.5],
         [5.8, 3.1, 5.0, 1.7]], dtype=np.float32
    )

    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": new_samples},
        num_epochs=1,
        shuffle=False
    )

    predictions = list(classifier.predict(input_fn=predict_input_fn))
    predicted_classes = [(p["classes"])[0] for p in predictions]

    
    print(
        "New Samples, Class Predictions: {}\n"
            .format(predicted_classes)
    )
    
    for p_class in predicted_classes:
        print(p_class)

In [31]:
main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_session_config': None, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_model_dir': './models/iris_model'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./models/iris_model/model.ckpt-14000
INFO:tensorflow:Saving checkpoints for 14001 into ./models/iris_model/model.ckpt.
INFO:tensorflow:step = 14001, loss = 8.98668
INFO:tensorflow:global_step/sec: 692.342
INFO:tensorflow:step = 14101, loss = 4.14046 (0.143 sec)
INFO:tensorflow:global_step/sec: 668.799
INFO:tensorflow:step = 14201, loss = 4.43707 (0.150 sec)
INFO:tensorflow:global_step/sec: 683.46
INFO:tensorflow:step = 14301, loss = 1.28895 (0.146 sec)
INFO:tensorflow:global_step/sec: 665.759
INFO:tensorflow:step = 14401, loss = 3.70411 (0.150 sec)
INFO:tensorflow: